In [1]:
! pip install datasets xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import xgboost as xgb
from datasets import load_dataset
from sklearn.metrics import accuracy_score
import numpy as np

#لود کردن دیتاست web 10k from HuggingFace

In [3]:
#https://huggingFace.co/datasets/philipphager/MSLR-WEB10k

dataset=load_dataset("philipphager/MSLR-WEB10k")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train.parquet:   0%|          | 0.00/260M [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/85.0M [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/86.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

#اماده سازی داده ها

داده های ورودی و برچسب هارو جدا میکنیم فرض بر این است که این دیتاست دارای ویژگی ها و برچسب هاست

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['query', 'labels', 'features', 'n'],
        num_rows: 6000
    })
    validation: Dataset({
        features: ['query', 'labels', 'features', 'n'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['query', 'labels', 'features', 'n'],
        num_rows: 2000
    })
})

In [5]:
sampel_train_data=dataset["train"].select(range(50))
sampel_test_data=dataset["test"].select(range(10))

In [6]:
x_train = sampel_train_data["features"]
y_train = sampel_train_data["labels"]

x_test = sampel_test_data["features"]
y_test = sampel_test_data["labels"]

In [7]:
len(x_train[0]) , len(x_train[1]), len(x_train[2])

#نشون میده برای اون کوری چندتا سند برگردونده شده

(86, 103, 111)

In [8]:
len(y_train[0]), len(y_train[1]), len(y_train[2])

#تعداد لیبل هایی که به ازای هر سند داریم

(86, 103, 111)

In [9]:
x_train_np = np.array(x_train,dtype=object)
y_train_np = np.array(y_train,dtype=object)

x_test_np = np.array(x_test,dtype=object)
y_test_np = np.array(y_test,dtype=object)

In [10]:
tmp= dataset['train'].select(range(2))
fea = tmp["features"]
labels=tmp["labels"]

In [11]:
len(fea[0])               #به ازای سوال اول هشتادو شیش تا سند داریم

86

In [13]:
len(labels[0])              #به ازای 86 تا سند 86 تا لیبل  داریم

86

In [12]:
tmp

Dataset({
    features: ['query', 'labels', 'features', 'n'],
    num_rows: 2
})

تعداد فیچر ها به ازای هر سند 136 تا است

In [14]:
x_train= np.vstack(x_train_np)
y_train= np.concatenate(y_train_np)

x_test= np.vstack(x_test_np)
y_test= np.concatenate(y_test_np)

#تبدیل داده ها به فرمت دی ماتریکس برای ایکس جی بوست

In [15]:
#هرداده ای که به تابع ترین ایکس جی بوست میدیم باید فرمت دی ماتریکس داشته باشه
#ورودی دی ماتریکس هم باید ماتریکس از نامپای باشه


dtrain= xgb.DMatrix(x_train,label=y_train)
dtest= xgb.DMatrix(x_test,label=y_test)

#تنظیمات هایپر پارامتر ها

In [16]:
params= {
    'objective': "multi:softmax",                   #فرض بر اینکه ما یک مسءله طبقه بندی چندکلاسه داریم
    'num_class': len(set(y_train)),                   #تعداد کلاس ها بر اساس  داده های اموزشی
    "max_depth": 6,                       #عمق درخت
    "eta" : 0.3,                                #نرخ یادگیری
    'eval_metric': "mlogloss"                   #معیار ارزیابی
}

#تعداد تکرار ها
num_rounds= 100

#اموزش مدل

In [17]:
bst=xgb.train(params,dtrain,num_rounds)

#پیش بینی رو داده تست

In [18]:
y_pred = bst.predict(dtest)

#محاسبه دقت

In [ ]:
accuracy= accuracy_score(y_test, y_pred)
print(f"{accuracy*100:.2f}%")